In [1]:
import importlib

import model
import pan_loader
import base_config
import loss_functions as L

import utils

import torch
import torch.nn as nn
import torch.optim as optim

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"


In [2]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

train_img_dir = val_img_dir 
train_seg_dir = val_seg_dir 
train_ann_json = val_ann_json 

In [3]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)

In [4]:
config = base_config.Config()

In [5]:
train_loader = pan_loader.get_loader(train_img_dir, train_seg_dir, train_ann, config)
val_loader = pan_loader.get_loader(val_img_dir, val_seg_dir, val_ann, config)

In [6]:
net = model.hgmodel()

In [7]:
def set_trainable(module,state):
    for param in module.parameters():
        param.requires_grad = state

set_trainable(net,False)

set_trainable(net.mb0,True)
set_trainable(net.mb1,True)
set_trainable(net.cb,True)

set_trainable(net.iresnet0, True)
set_trainable(net.iresnet1, True)

# for name,child in net.iresnet0.named_children():
#     if name[:-1] == "layer":
#         [set_trainable(s.ignore_filters,True) for s in child[::2]]
#         [set_trainable(s.copy_filters,True) for s in child[::2]]

param_lr = []
param_lr.append({'params': net.mb0.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.mb1.parameters(),'lr':1e-3,'momentum':0.9})
param_lr.append({'params': net.cb.parameters(),'lr':1e-3,'momentum':0.9})

# for name,child in net.iresnet0.named_children():
#     if name[:-1] == "layer":
#         for s in child[::2]:
#             lr = 0
#             if int(name[-1]) > 0:
#                 lr = 1e-3
#             else:
#                 lr = 0
#             param_lr.append({'params':s.ignore_filters.parameters(),'lr':lr,'momentum':0.9})
#             param_lr.append({'params':s.copy_filters.parameters(),'lr':1e-3,'momentum':0.9})

param_lr = [{'params':net.parameters(),'lr':1e-2, 'momentum':0.9}]
net_size = sum([i.numel() for i in net.parameters()])
trainable_params = filter(lambda p: p.requires_grad, net.parameters())
trainable_size = sum([i.numel() for i in trainable_params])
print(net_size,trainable_size)
print(param_lr)
optimizer = optim.SGD(param_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=100, min_lr=1e-2, eps=1e-08)

69912050 69912050
[{'params': <generator object Module.parameters at 0x7fe3e7f29620>, 'lr': 0.01, 'momentum': 0.9}]


In [8]:
# net = nn.DataParallel(net, device_ids=[0,1])
net = net.cuda()

In [9]:
ckpt = utils.Checkpoint(iters_per_epoch=60, model_dir="./models/", model_name="first")

for i, data in enumerate(train_loader,0):
#     print("batch %d:"%i)
    optimizer.zero_grad()
    
    images, impulses, instance_masks, cat_ids = utils.cudify_data(data)
    print(cat_ids)
    del(data)
    outs = net([images,impulses])
    del(images, impulses)
    loss = L.loss_criterion1(outs, [instance_masks, cat_ids])
    del(instance_masks, cat_ids, outs)
    ckpt.update(loss.data, net)
    loss.backward()
    del(loss)
    optimizer.step()

[1]
[ 22 131] [131]
[1]
[17 42] [42]
[3]
[  1   1  21 117 118 120 127 130 132] [117]
[6]
[  1   1   1  33 101 118 124 127 132] [124]
[4]
[ 16  62 112 121 123 128 132 133] [123]
[3]
[ 42  55  61  61 122 132] [61]
[4]
[  1   3  11 101 124] [124]
[0]
[  1  54  57  87 122 123 132] [1]
[14]
[  1   1   1  27  40  54  54  61  61 110 113 116 122 124 128 129] [128]
[11]
[ 21  21  21  21  21  21  21  21  21  21 100 131] [131]
[10]
[ 40  40  40  40  40  40  40  40  62 123 132] [132]
[6]
[  1   6  10  10  10 117 130 132] [130]
[13]
[ 40  40  62  72  85  86  87  93  94 112 119 121 123 132] [132]
[0]
[14 16 16 88 94] [14]
[tensor([131], device='cuda:0')]
torch.Size([1, 1, 448, 448]) torch.Size([1, 1, 448, 448]) torch.Size([1]) torch.Size([1, 134])
tensor(0.7741, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.8414, device='cuda:0', grad_fn=<MeanBackward1>)
[tensor([117], device='cuda:0')]
[2]
[ 43  44 129] [129]
[3]
[  1   1  42  60  60  82 116 122 123 128] [60]
[2]
[ 21  21 117 126] [117]
[4]
[ 

[tensor([7], device='cuda:0')]
torch.Size([1, 1, 448, 448]) torch.Size([1, 1, 448, 448]) torch.Size([1]) torch.Size([1, 134])
tensor(0.7834, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.9484, device='cuda:0', grad_fn=<MeanBackward1>)
[tensor([132], device='cuda:0')]
[12]
[  1   1  25  42  42  54  54  54  59  61 122 128 132] [132]
[12]
[  1   1   1   1   1   1  21  21  21  21  21  21 100 117] [100]
[13]
[ 57  58  61  74  74  74  74  76  76  76  89  96 122 132] [132]
[2]
[47 47 61] [61]
torch.Size([1, 1, 448, 448]) torch.Size([1, 1, 448, 448]) torch.Size([1]) torch.Size([1, 134])
tensor(0.8350, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.5627, device='cuda:0', grad_fn=<MeanBackward1>)
[tensor([129], device='cuda:0')]
torch.Size([1, 1, 448, 448]) torch.Size([1, 1, 448, 448]) torch.Size([1]) torch.Size([1, 134])
tensor(0.4878, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.9652, device='cuda:0', grad_fn=<MeanBackward1>)
[8]
[ 24  24  87 107 116 118 119 123 132] [132]
[tens

[15]
[  1   1   1   1   1   1   1   1   1   1   1   1   1 120 124 130] [130]


Process Process-15:
Process Process-8:
Process Process-14:
Process Process-4:
Process Process-11:
Process Process-13:
Process Process-1:
Process Process-16:
Process Process-10:
Process Process-12:
Process Process-7:
Process Process-2:
Process Process-6:
Process Process-9:
Process Process-5:
Traceback (most recent call last):
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):


[tensor([106], device='cuda:0')]
torch.Size([1, 1, 448, 448]) torch.Size([1, 1, 448, 448]) torch.Size([1]) torch.Size([1, 134])
tensor(0.7386, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.7727, device='cuda:0', grad_fn=<MeanBackward1>)
[tensor([61], device='cuda:0')]
torch.Size([1, 1, 448, 448]) torch.Size([1, 1, 448, 448]) torch.Size([1]) torch.Size([1, 134])
tensor(0.2685, device='cuda:0', grad_fn=<MeanBackward1>) tensor(4.4871, device='cuda:0', grad_fn=<MeanBackward1>)


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._targe

  File "/home/aravind/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 110, in _worker_loop
    data_queue.put((idx, samples))
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/aravind/anaconda3/lib/python3.6/multiprocessing/queues

KeyboardInterrupt: 